# Install transformers and mount drive

In [1]:
pip install transformers


     |████████████████████████████████| 2.3MB 3.9MB/s 
     |████████████████████████████████| 3.3MB 39.3MB/s 
     |████████████████████████████████| 901kB 46.2MB/s 


In [2]:
from transformers import pipeline 
print(pipeline('sentiment-analysis')('we love you'))


[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Imports


In [31]:
import transformers as ppb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

#for sentence preprocessing
import string
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Get And Preprocess Data

In [42]:
dataRaw = pd.read_excel('/content/gdrive/MyDrive/Praxisprojekt/Trainingdata.xlsx', sheet_name = 'sentences')

dataRaw.drop(['SUBJindl', 'SUBJsrce', 'SUBJrhet', 'SUBJster', 'SUBJspee', 'SUBJinspe', 'SUBJprop', 'SUBJpolit'], axis=1, inplace=True)

data = dataRaw['Sentence'].tolist()
data = list((str(s) for s in data))

#data2 mit blatant kopiertem preprocessing von marlene: all lowercase -> punctuation raus -> zahlen raus -> stopwords raus
data2 = dataRaw['Sentence'].tolist()
data2 = list((str(s) for s in data2))
data2 = list((s.lower() for s in data2))
table = str.maketrans('', '', string.punctuation)
data2 = [s.translate(table) for s in data2]
data2 = [re.sub(r'\d+', '', s) for s in data2]
stopwords = set(nltk.corpus.stopwords.words('english'))
data2 = [' '.join([word for word in s.split() if word not in stopwords]) for s in data2]
print(data2)

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

#-------------------------------------------------------------------------------------------
def prepareForBert(sentences):
#Sätze zerstückeln lassen
  for s in sentences:
    tokenizer.encode(s, add_special_tokens=True)

#Padding hinzufügen 
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  #Maske erstellen, um das Padding bei der Verarbeitung zu filtern
  mask = np.where(padded != 0, 1, 0)
  mask.shape

  #mache padded Array und Maske zu einem Tensor
  #Tensor = mehrdimensionale Matrix mit einheitlichem Datentyp
  input = torch.tensor(padded)  
  mask = torch.tensor(mask)
  return input, mask
#--------------------------------------------------------------------------------------------

#bereite 2 Datensätze vor
input1, mask1 = prepareForBert(data)
print("Set 1 fertig")
input2, mask2 = prepareForBert(data2)
print("Set 2 fertig")

['miley liam fighting false rumors swirl theyre feud supposed prenup', 'miley cyrus liam hemsworth fighting', 'new report claims cyrus hemsworth huge fight supposed prenup making wedding plans', 'according report cyrus hemsworth planning wedding himalayan mountains summer', 'reportedly tension prenup', 'cyrus reportedly told hemsworth wanted sign prenup got married could protect estimated million net worth', 'hemsworth reportedly shocked annoyed miley accused trusting', 'miley liam getting married summer click details', 'sources say miley cyrus liam hemsworth fighting prenup photo splash news', 'problem', 'report true', 'sources told gossip cop hasnt drama cyrus hemsworth prenup', 'cyrus hemsworth dont even plans get married himalayan mountains summer', 'far first time cyrus hemsworth faced false rumors relationship theyve faced constant breakup baby wedding rumors since got back together', 'miley liams wedding update', 'last month cyrus hemsworth faced rumors fighting yet', 'time sour

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Set 1 fertig


# Lasse Bert von der Leine (Läuft lange)


In [ ]:
#gib unser Zeug an BERT
#no_grad = Angabe zur Simplifikation des Rechenvorgangs
with torch.no_grad():
    output = model(input, attention_mask=mask)

#nur die erste Spalte auslesen = von BERT geschriebene Kennwerte
features = output[0][:,0,:].numpy()

#Füttere SciKit mit Berts Daten

In [ ]:
#labels holen
labels = dataDumb[['SUBJlang', 'SUBJopin']]

label1 = labels['SUBJlang']
label2 = labels['SUBJopin']

#und jetzt alles durch die Toolbox von Scikit jagen
train_features, test_features, train_labels, test_labels = train_test_split(features, label1)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

#Score
print("Score SUBJlang")
print(lr_clf.score(test_features, test_labels))

#und jetzt NOCHMAL alles durch die Toolbox von Scikit jagen
train_features, test_features, train_labels, test_labels = train_test_split(features, label2)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

#Score 2
print("Score SUBJopin")
print(lr_clf.score(test_features, test_labels))

#und JETZT Naive Bayes

train_features, test_features, train_labels, test_labels = train_test_split(features, label1)

nb_clf = GaussianNB()
nb_clf.fit(train_features, train_labels)

print("Score: Naive Bayes")
print("SUBJlang")
print(nb_clf.score(test_features, test_labels))

train_features, test_features, train_labels, test_labels = train_test_split(features, label2)

nb_clf = GaussianNB()
nb_clf.fit(train_features, train_labels)

print("SUBJopin")
print(nb_clf.score(test_features, test_labels))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Score SUBJlang
0.6070796460176991
Score SUBJopin
0.7256637168141593
Score: Naive Bayes
SUBJlang
0.46017699115044247
SUBJopin
0.5451327433628319


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
import pandas as pd
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df = df.head(3)
print(df[0])

0    a stirring , funny and finally transporting re...
1    apparently reassembled from the cutting room f...
2    they presume their audience wo n't sit still f...
Name: 0, dtype: object
